In [1]:
!pip install --upgrade openai wikipedia-api serpapi requests gtts onnx onnxruntime huggingface_hub streamlit amadeus
!pip install --upgrade langchain-openai
!pip install --upgrade google-search-results

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=aa80793cbf7bf98be793c6df6e7636d

In [2]:
from moviepy.editor import *

  if event.key is 'enter':



In [4]:
from serpapi import GoogleSearch
print("✅ GoogleSearch is imported successfully!")


✅ GoogleSearch is imported successfully!


In [5]:
import openai
import wikipediaapi
import serpapi
import requests
import gtts
import moviepy
import onnx
import onnxruntime
import huggingface_hub
import streamlit
import amadeus

print("✅ All libraries are installed successfully!")


✅ All libraries are installed successfully!


In [6]:
# Create necessary files
!touch app.py travel_story.py config.py utils.py


# **Write utils.py (Helper Functions for API Calls)**

In [9]:
%%writefile utils.py

import requests
from config import GOOGLE_MAPS_API_KEY, WEATHER_API_KEY, AMADEUS_API_KEY

# ✅ Function to fetch latitude & longitude
def get_lat_lng(location):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": location, "key": GOOGLE_MAPS_API_KEY}
    response = requests.get(url, params=params).json()
    if "results" in response and response["results"]:
        location_data = response["results"][0]["geometry"]["location"]
        return location_data["lat"], location_data["lng"]
    return None, None

# ✅ Function to fetch tourist attractions
def fetch_tourist_attractions(location, top_n=5):
    lat, lng = get_lat_lng(location)
    if not lat or not lng:
        return "Could not determine the exact location."

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lng}", "radius": 10000, "type": "tourist_attraction", "key": GOOGLE_MAPS_API_KEY}

    response = requests.get(url, params=params).json()
    if "results" in response:
        return [f"{t['name']} ({t.get('rating', 'No rating')}⭐)" for t in response["results"][:top_n]]
    return "No tourist attractions found."

# ✅ Function to fetch restaurants
def fetch_restaurants(location, purpose, top_n=5):
    lat, lng = get_lat_lng(location)
    if not lat or not lng:
        return "Could not determine the exact location."

    keyword = {
        "Leisure": "casual dining",
        "Business": "fine dining",
        "Family": "family-friendly",
        "Adventure": "unique cuisine",
        "Romantic": "romantic restaurant"
    }.get(purpose, "restaurant")

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lng}", "radius": 5000, "type": "restaurant", "keyword": keyword, "key": GOOGLE_MAPS_API_KEY}

    response = requests.get(url, params=params).json()
    if "results" in response:
        return [f"{r['name']} ({r.get('rating', 'No rating')}⭐)" for r in response["results"][:top_n]]
    return "No restaurants found."

# ✅ Function to fetch real-time weather details
def fetch_weather(city):
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": WEATHER_API_KEY, "units": "metric"}
    response = requests.get(url, params=params).json()
    if "weather" in response and "main" in response:
        return f"{response['weather'][0]['description'].capitalize()}, {response['main']['temp']}°C"
    return "Weather data not available."

# ✅ Function to fetch flight details
def fetch_flight_details(origin, destination, departure_date):
    try:
        response = requests.get(f"https://api.flightapi.com/{origin}/{destination}/{departure_date}", headers={"API-Key": AMADEUS_API_KEY})
        if response.status_code == 200:
            return response.json()
        return "Flight details unavailable. Please try again later."
    except Exception as e:
        return f"Error retrieving flight details: {str(e)}"

# ✅ Function to fetch hotels
def fetch_hotels(location, top_n=5):
    lat, lng = get_lat_lng(location)
    if not lat or not lng:
        return "Could not determine the exact location."

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lng}", "radius": 5000, "type": "lodging", "key": GOOGLE_MAPS_API_KEY}

    response = requests.get(url, params=params).json()
    if "results" in response:
        return [f"{h['name']} ({h.get('rating', 'No rating')}⭐)" for h in response["results"][:top_n]]
    return "No hotels found."


Overwriting utils.py


In [38]:
!pip install moviepy
from moviepy.editor import ImageSequenceClip, concatenate_videoclips, AudioFileClip, CompositeVideoClip


# **Write travel_story.py (AI-Powered Travel Story & Video Generation)**

In [12]:
%%writefile travel_story.py
import openai
import requests
import wikipediaapi
import io
import time
import onnx
import numpy as np
from PIL import Image
from moviepy import *  # For video editing
from gtts import gTTS  # For text-to-speech
from onnxruntime import InferenceSession
from transformers import AutoModelForCausalLM, AutoTokenizer
from config import llm, OPENAI_API_KEY # Import the latest API key from config.py
import openai

# ✅ Ensure OpenAI API is using the latest key
openai.api_key = OPENAI_API_KEY  # This ensures the latest key is used every time

# ✅ Initialize OpenAI Client
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# ✅ Wikipedia API Setup
wiki = wikipediaapi.Wikipedia(user_agent="MyTravelApp/1.0", language="en")

# ✅ Function to fetch travel data from Wikipedia
def get_wikipedia_summary(place):
    page = wiki.page(place)
    return page.summary[:500] if page.exists() else "No Wikipedia summary found."

def generate_travel_story(destination, purpose, start_date, end_date):
    wikipedia_info = get_wikipedia_summary(destination)

    purpose_templates = {
        "leisure": f"You are about to explore {destination} on a relaxing leisure trip from {start_date} to {end_date}. Describe your experiences as you visit famous landmarks, stroll through parks, and enjoy the peaceful atmosphere.",
        "food": f"As a food lover, you're in {destination} from {start_date} to {end_date} to explore its delicious street food, high-end restaurants, and unique local flavors. Describe the dishes you'll taste, the bustling food markets, and the famous cafés you’ll visit.",
        "adventure": f"You're visiting {destination} for an adrenaline-filled adventure from {start_date} to {end_date}. Describe thrilling activities such as hiking, surfing, skydiving, and other outdoor experiences in the area.",
        "business": f"You're traveling to {destination} for a business trip from {start_date} to {end_date}. Describe your meetings, networking events, and the city's corporate atmosphere. Also, mention any work-life balance experiences, like after-hours dining or sightseeing.",
        "romantic": f"You're in {destination} for a romantic getaway from {start_date} to {end_date}. Describe the intimate dinners, scenic walks, and breathtaking sunset views you'll experience with your partner.",
        "spiritual": f"You're visiting {destination} for a spiritual retreat from {start_date} to {end_date}. Describe the meditation spots, temples, churches, and peaceful landscapes where you'll find tranquility and reflection.",
        "family": f"You're on a family trip to {destination} from {start_date} to {end_date}, filled with fun and bonding moments. Describe the amusement parks, kid-friendly attractions, and the joy of exploring new places together."
    }

    purpose_prompt = purpose_templates.get(purpose, purpose_templates["leisure"])

    full_prompt = f"""
    {purpose_prompt}

    Be immersive, engaging, and detailed. Use vivid descriptions and include unique aspects of {destination}.

    Wikipedia Summary: {wikipedia_info}

    Travel Story:
    """

    response = llm.invoke(full_prompt)
    return response.content if hasattr(response, 'content') else str(response)
# ✅ Function to generate a travel plan
def generate_travel_plan(destination, start_date, end_date, purpose):
    prompt = f"""
    Generate a detailed travel itinerary for a trip to {destination} from {start_date} to {end_date} for {purpose}.
    Include accommodations, food recommendations, attractions, and transportation details.
    """
    response = llm.invoke(prompt)
    return response.content if hasattr(response, 'content') else str(response)

# ✅ Function to generate exactly 5 travel images based on purpose with timeout & retry
def generate_travel_images(destination, purpose):
    prompt_templates = {
        "leisure": [
            f"A breathtaking aerial view of {destination} at sunset, ultra-HD",
            f"A famous landmark in {destination} with tourists exploring",
            f"A peaceful park or nature spot in {destination}",
            f"A vibrant street with people walking in {destination}",
            f"A scenic sunset view from a rooftop in {destination}"
        ],
        "food": [
            f"A vibrant street food market in {destination}, filled with people enjoying local delicacies",
            f"A famous restaurant in {destination} serving authentic dishes",
            f"A cozy café in {destination}, perfect for food lovers",
            f"A chef cooking a signature dish in {destination}",
            f"A table full of delicious traditional food in {destination}"
        ],
        "adventure": [
            f"An exciting mountain hiking trail in {destination}, breathtaking views",
            f"A water sport activity like surfing or rafting in {destination}",
            f"A deep forest trekking scene in {destination}, wild nature",
            f"A scenic road trip along the coast in {destination}",
            f"A person skydiving over {destination}, adrenaline rush"
        ],
        "business": [
            f"A futuristic business district in {destination} with skyscrapers",
            f"A conference hall with professionals networking in {destination}",
            f"A co-working space with laptops and professionals working in {destination}",
            f"A businessperson having a coffee meeting in {destination}",
            f"A night view of {destination} with lights from corporate buildings"
        ],
        "romantic": [
            f"A romantic couple watching the sunset in {destination}, cinematic lighting",
            f"A candlelit dinner with a scenic view in {destination}",
            f"A dreamy evening walk in {destination}, beautiful city lights",
            f"A couple holding hands in front of a famous monument in {destination}",
            f"A cozy picnic in a park at {destination}, romantic atmosphere"
        ],
        "spiritual": [
            f"A peaceful Buddhist temple in {destination}, monks meditating",
            f"A yoga retreat in {destination} surrounded by nature",
            f"A quiet and scenic meditation spot in {destination}",
            f"A historical church or mosque in {destination}, spiritual atmosphere",
            f"A sunrise view from a hill in {destination}, peaceful and calm"
        ],
        "family": [
            f"A fun amusement park in {destination}, families enjoying rides",
            f"A zoo or aquarium in {destination}, children looking at animals",
            f"A large family picnic in {destination}, happy and cheerful atmosphere",
            f"A family exploring a famous museum in {destination}",
            f"A scenic beach in {destination} with kids playing in the sand"
        ],
    }

    selected_prompts = prompt_templates.get(purpose, prompt_templates["leisure"])
    image_urls = []

    for prompt in selected_prompts[:5]:  # Ensure we get exactly 5 images

        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            n=1,
            size="1024x1024"

        )

        image_urls.append(response.data[0].url)
    return image_urls


# ✅ Function to create a voice-over for the travel story
def generate_voiceover(story_text, output_audio="travel_narration.mp3"):
    tts = gTTS(text=story_text, lang="en", slow=False)
    tts.save(output_audio)
    return output_audio

def create_travel_video(image_urls, narration_audio, output_video="travel_story.mp4"):
    audio_clip = AudioFileClip(narration_audio)
    total_audio_duration = audio_clip.duration
    image_duration = total_audio_duration / 5  # Divide equally among 5 images

    image_clips = []
    for i, url in enumerate(image_urls):
        response = requests.get(url)
        image = Image.open(io.BytesIO(response.content))
        image_path = f"travel_image_{i}.jpg"
        image.save(image_path)

        clip = ImageClip(image_path, duration=image_duration).set_fps(24)
        clip = clip.resize(lambda t: 1 + 0.01 * t)  # Slow zoom-in effect
        image_clips.append(clip)

    video_clip = concatenate_videoclips(image_clips, method="compose")
    video_clip = video_clip.set_audio(audio_clip)
    video_clip.write_videofile(output_video, codec="libx264", fps=24, audio_codec="aac")
    print("🎬 Travel story animation created successfully!")


# ✅ Execution for Testing
if __name__ == "__main__":
    destination = "Paris"
    purpose = "Leisure"
    start_date = "2025-04-10"
    end_date = "2025-04-20"

    print("📅 Generating Travel Plan...")
    travel_plan = generate_travel_plan(destination, start_date, end_date, purpose)
    print(f"📝 Travel Plan:\n{travel_plan}")

    print("🔄 Fetching data and generating travel story...")
    travel_story_text = generate_travel_story(destination, purpose, start_date, end_date)
    print(f"📖 Travel Story:\n{travel_story_text}")

    print("🖼 Generating Travel Images...")
    image_urls = generate_travel_images(destination, purpose)
    print(f"Generated Images: {image_urls}")

    print("🎤 Generating voiceover...")
    narration_file = generate_voiceover(travel_story_text)

    print("🎥 Creating travel video...")
    travel_video = create_travel_video(image_urls, narration_file)
    print(f"✅ Travel video saved as {travel_video}")


📅 Generating Travel Plan...
📝 Travel Plan:
Day 1: Arrival in Paris (2025-04-10)
- Arrival at Charles de Gaulle Airport.
- Transfer to Hotel Le Bristol Paris via private taxi.
- Rest and freshen up.
- Evening stroll around the local area.
- Dinner at the hotel's Michelin-starred restaurant, Epicure.

Day 2: Exploring Paris (2025-04-11)
- Breakfast at the hotel.
- Visit the Louvre Museum. Pre-booked tickets for a guided tour.
- Lunch at Café Marly overlooking the Louvre.
- Visit Notre-Dame Cathedral.
- Dinner at Le Comptoir du Relais in the Saint-Germain district.

Day 3: More of Paris (2025-04-12)
- Breakfast at the hotel.
- Visit the Eiffel Tower. Pre-booked tickets for the summit.
- Lunch at Le Jules Verne, located on the Eiffel Tower.
- Visit the Musée d'Orsay.
- Dinner at L'Ambroisie in the Marais district.

Day 4: Day Trip to Versailles (2025-04-13)
- Breakfast at the hotel.
- Day trip to the Palace of Versailles. Pre-booked tickets for a guided tour.
- Lunch at La Petite Venise in

MoviePy - Done.
Moviepy - Writing video travel_story.mp4



Moviepy - Done !
Moviepy - video ready travel_story.mp4
🎬 Travel story animation created successfully!
✅ Travel video saved as None


# **Write app.py (Streamlit UI)**

In [40]:
%%writefile app.py
import streamlit as st
from travel_story import generate_travel_story, generate_voiceover, create_travel_video, generate_travel_plan, generate_travel_images
from utils import fetch_weather, fetch_tourist_attractions, fetch_flight_details, fetch_restaurants, fetch_hotels
import requests
from PIL import Image
from io import BytesIO
from moviepy.editor import AudioFileClip

st.set_page_config(page_title="✈️ AI Travel Planner", layout="wide")

st.title("✈️ AI Travel Planner 🏨")

st.sidebar.title("Plan Your Trip 🗺")
destination = st.sidebar.text_input("Enter Destination", "Paris")
start_date = st.sidebar.date_input("Start Date")
end_date = st.sidebar.date_input("End Date")
purpose = st.sidebar.selectbox("Purpose of Visit", ["Leisure", "Business", "Adventure", "Romantic", "Family"])

if st.sidebar.button("Generate Travel Plan & Details"):
    with st.spinner("🔄 Generating AI travel plan & fetching details..."):
        travel_plan_text = generate_travel_plan(destination, start_date, end_date, purpose)
        weather_info = fetch_weather(destination)
        tourist_attractions = fetch_tourist_attractions(destination)
        restaurants = fetch_restaurants(destination, purpose)
        hotels = fetch_hotels(destination)
        flights = fetch_flight_details("JFK", destination, start_date)

    st.subheader("📅 Your AI-Generated Travel Plan")
    st.write(travel_plan_text)

    st.subheader("🌦 Weather Forecast")
    st.write(weather_info)

    st.subheader("🏛 Tourist Attractions")
    st.write(tourist_attractions)

    st.subheader("🍽 Recommended Restaurants")
    st.write(restaurants)

    st.subheader("🏨 Hotel Recommendations")
    st.write(hotels)

    st.subheader("✈️ Flight Details")
    st.write(flights)

if st.sidebar.button("Generate Story & Voiceover"):
    with st.spinner("🔄 Generating AI travel story..."):
        travel_story_text = generate_travel_story(destination, start_date, end_date, purpose)
        narration_audio = generate_voiceover(travel_story_text)

    st.subheader("📖 Your AI-Generated Travel Story")
    st.write(travel_story_text)

    st.subheader("🎤 AI Voiceover")
    st.audio(narration_audio)

if st.sidebar.button("Generate Images & Video"):
    with st.spinner("🖼 Generating Travel Images..."):
        travel_images = generate_travel_images(destination, purpose)

    st.subheader("🖼 View Destination Images")
    if travel_images:
        for image_url in travel_images:
            response = requests.get(image_url)
            if response.status_code == 200:
                image = Image.open(BytesIO(response.content))
                st.image(image, caption=f"A view of {destination}", use_column_width=True)
            else:
                st.warning("❌ Unable to fetch image. Try again later.")
    else:
        st.warning("❌ No images generated.")

    with st.spinner("🎥 Creating AI Travel Video..."):
        travel_story_text = generate_travel_story(destination, start_date, end_date, purpose)
        narration_audio = generate_voiceover(travel_story_text)
        travel_video = create_travel_video(travel_images, narration_audio)

    st.subheader("🎥 AI-Generated Travel Video")
    if travel_video:
        st.video(travel_video)
    else:
        st.warning("❌ Video generation failed.")




Overwriting app.py


In [41]:
%%writefile verify_imports.py

import os

# ✅ Check if all necessary files exist
required_files = ["config.py", "utils.py", "travel_story.py", "app.py"]
missing_files = [file for file in required_files if not os.path.exists(file)]

if missing_files:
    print(f"❌ ERROR: Missing files: {missing_files}. Ensure all required files are present.")
else:
    print("✅ All necessary files exist.")

# ✅ Verify `config.py` imports
try:
    from config import google_maps_api_key, serpapi_key, WEATHER_API_KEY
    print("✅ Successfully imported API keys from config.py")
    print(f"Google Maps API Key: {google_maps_api_key[:5]}******")
    print(f"SerpAPI Key: {serpapi_key[:5]}******")
    print(f"Weather API Key: {weather_api_key[:5]}******")
except ModuleNotFoundError:
    print("❌ ERROR: 'config.py' not found.")
except ImportError:
    print("❌ ERROR: Could not import variables from 'config.py'.")

# ✅ Verify `utils.py` imports
try:
    from utils import get_lat_lng, fetch_restaurants, fetch_weather
    print("✅ Successfully imported functions from utils.py")
    print(f"get_lat_lng function exists: {callable(get_lat_lng)}")
    print(f"fetch_restaurants function exists: {callable(fetch_restaurants)}")
    print(f"fetch_weather function exists: {callable(fetch_weather)}")
except ModuleNotFoundError:
    print("❌ ERROR: 'utils.py' not found.")
except ImportError:
    print("❌ ERROR: Could not import functions from 'utils.py'.")

# ✅ Verify `purpose.py` imports
try:
    from travel_story import generate_travel_story, generate_voiceover, create_travel_video
    print("✅ Successfully imported functions from purpose.py")
    print(f"generate_travel_story function exists: {callable(generate_travel_story)}")
    print(f"generate_voiceover function exists: {callable(generate_voiceover)}")
    print(f"create_travel_video function exists: {callable(create_travel_video)}")
except ModuleNotFoundError:
    print("❌ ERROR: 'purpose.py' not found.")
except ImportError:
    print("❌ ERROR: Could not import functions from 'purpose.py'.")

# ✅ Verify `app.py` existence
if os.path.exists("app.py"):
    print("✅ 'app.py' exists and is ready to run.")
else:
    print("❌ ERROR: 'app.py' is missing.")


Overwriting verify_imports.py


In [42]:

!pip install pyngrok

from pyngrok import ngrok
!kill $(pgrep -f ngrok)

# Run Streamlit app
!streamlit run app.py &>/content/logs.txt &
# Set up ngrok
!ngrok authtoken 2suOuuxs3zjz3pWQWDl9dQZPTLR_5FKMVwXpnfPJDKgESoGpK

public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")

^C
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://31fc-35-184-14-88.ngrok-free.app" -> "http://localhost:8501"
